In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, size

spark = SparkSession.builder.getOrCreate()

def read_images(split_name):
    """
    Reads images from the specified split directory and extracts breed labels.

    Args:
        split_name (str): The data split to read ('train', 'val', 'test').

    Returns:
        DataFrame: A Spark DataFrame containing image binary data and breed labels.
    """
    path = f"dbfs:/mnt/dog_datasets/{split_name}/"
    print(f"Reading images from path: {path}")
    
    df = spark.read.format("binaryFile") \
        .option("pathGlobFilter", "*.jpg") \
        .option("recursiveFileLookup", "true") \
        .load(path)
    
    print("Schema of the loaded DataFrame:")
    df.printSchema()
    
    df = df.withColumn("path_parts", split(col("path"), "/"))
    df = df.withColumn("breed", col("path_parts").getItem(size(col("path_parts")) - 2))
    df = df.drop("path_parts")
    
    print("Sample data after extracting breed:")
    df.select("path", "breed").show(5, truncate=False)
    
    return df
train_df = read_images("train")


Reading images from path: dbfs:/mnt/dog_datasets/train/
Schema of the loaded DataFrame:
root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)

Sample data after extracting breed:


/databricks/spark/python/pyspark/sql/column.py:486: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


+------------------------------------------------------------------------------+------------------------------+
|path                                                                          |breed                         |
+------------------------------------------------------------------------------+------------------------------+
|dbfs:/mnt/dog_datasets/train/n02085620-Chihuahua/n02085620_4602.jpg           |n02085620-Chihuahua           |
|dbfs:/mnt/dog_datasets/train/n02091635-otterhound/n02091635_1590.jpg          |n02091635-otterhound          |
|dbfs:/mnt/dog_datasets/train/n02106382-Bouvier_des_Flandres/n02106382_3756.jpg|n02106382-Bouvier_des_Flandres|
|dbfs:/mnt/dog_datasets/train/n02093647-Bedlington_terrier/n02093647_2743.jpg  |n02093647-Bedlington_terrier  |
|dbfs:/mnt/dog_datasets/train/n02107142-Doberman/n02107142_8331.jpg            |n02107142-Doberman            |
+------------------------------------------------------------------------------+------------------------

In [ ]:
def generate_label_mapping(spark_df):
    unique_breeds = spark_df.select("breed").distinct().rdd.flatMap(lambda x: x).collect()
    unique_breeds_sorted = sorted(unique_breeds)  
    
    label_mapping = {breed: idx for idx, breed in enumerate(unique_breeds_sorted)}
    
    return label_mapping

label_mapping = generate_label_mapping(train_df)

print("Label Mapping:")
for breed, idx in label_mapping.items():
    print(f"{breed}: {idx}")

print(f"Total classes: {len(label_mapping)}")


Label Mapping:
n02085620-Chihuahua: 0
n02085782-Japanese_spaniel: 1
n02085936-Maltese_dog: 2
n02086079-Pekinese: 3
n02086240-Shih-Tzu: 4
n02086646-Blenheim_spaniel: 5
n02086910-papillon: 6
n02087046-toy_terrier: 7
n02087394-Rhodesian_ridgeback: 8
n02088094-Afghan_hound: 9
n02088238-basset: 10
n02088364-beagle: 11
n02088466-bloodhound: 12
n02088632-bluetick: 13
n02089078-black-and-tan_coonhound: 14
n02089867-Walker_hound: 15
n02089973-English_foxhound: 16
n02090379-redbone: 17
n02090622-borzoi: 18
n02090721-Irish_wolfhound: 19
n02091032-Italian_greyhound: 20
n02091134-whippet: 21
n02091244-Ibizan_hound: 22
n02091467-Norwegian_elkhound: 23
n02091635-otterhound: 24
n02091831-Saluki: 25
n02092002-Scottish_deerhound: 26
n02092339-Weimaraner: 27
n02093256-Staffordshire_bullterrier: 28
n02093428-American_Staffordshire_terrier: 29
n02093647-Bedlington_terrier: 30
n02093754-Border_terrier: 31
n02093859-Kerry_blue_terrier: 32
n02093991-Irish_terrier: 33
n02094114-Norfolk_terrier: 34
n02094258-No

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),        
    transforms.RandomHorizontalFlip(),        
    transforms.ToTensor(),                    
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])  
])

val_test_transforms = transforms.Compose([
    transforms.Resize(256),                    
    transforms.CenterCrop(224),               
    transforms.ToTensor(),                     
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  
                         std=[0.229, 0.224, 0.225])   
])


class SparkImageDataset(Dataset):
    def __init__(self, spark_df, label_mapping, transform=None):
        """
        Initializes the dataset by collecting data from the Spark DataFrame.

        Args:
            spark_df (DataFrame): Spark DataFrame containing image data and labels.
            label_mapping (dict): Mapping from breed names to integer labels.
            transform (callable, optional): Transformations to apply to the images.
        """
        self.data = spark_df.select("content", "breed").collect()
        self.label_mapping = label_mapping
        self.transform = transform
        
        initial_length = len(self.data)
        self.data = [row for row in self.data if row.breed in self.label_mapping]
        filtered_length = len(self.data)
        print(f"Total samples in SparkImageDataset: {filtered_length} (Filtered out {initial_length - filtered_length})")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        """
        Retrieves the image and its corresponding label at the specified index.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            tuple: (image_tensor, label)
        """
        image_bytes = self.data[idx].content
        breed = self.data[idx].breed
        label = self.label_mapping[breed]
        
        image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label
print(len(label_mapping))
train_dataset = SparkImageDataset(train_df, label_mapping, transform=train_transforms)
val_dataset = SparkImageDataset(val_df, label_mapping, transform=val_test_transforms)
test_dataset = SparkImageDataset(test_df, label_mapping, transform=val_test_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Testing samples: {len(test_dataset)}")


120
Total samples in SparkImageDataset: 14355 (Filtered out 0)
Total samples in SparkImageDataset: 3025 (Filtered out 0)
Total samples in SparkImageDataset: 3200 (Filtered out 0)
Training samples: 14355
Validation samples: 3025
Testing samples: 3200


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
from torchmetrics import Accuracy
from tqdm import tqdm
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(ResNetClassifier, self).__init__()
        self.model = models.resnet50(pretrained=pretrained)
        
        for param in self.model.parameters():
            param.requires_grad = False
        
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)
        
    def forward(self, x):
        return self.model(x)
num_classes = len(label_mapping)

model = ResNetClassifier(num_classes=num_classes, pretrained=True).to(device)

print(model)


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNetClassifier(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
   

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
from torch.optim.lr_scheduler import StepLR

scheduler = StepLR(optimizer, step_size=7, gamma=0.1)
from tqdm import tqdm


num_epochs = 25
best_val_acc = 0.0
patience = 5  
epochs_no_improve = 0

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    print("-" * 10)
    
    
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train() 
            dataloader = train_loader
        else:
            model.eval()  
            dataloader = val_loader
        
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels in tqdm(dataloader, desc=f"{phase.capitalize()} Phase"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)           
                _, preds = torch.max(outputs, 1)   
                loss = criterion(outputs, labels)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)
        
        print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
        
        if phase == 'val':
            if epoch_acc > best_val_acc:
                best_val_acc = epoch_acc
                torch.save(model.state_dict(), "/tmp/best_resnet_model.pth")
                print("Best model updated.")
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                print(f"No improvement for {epochs_no_improve} epoch(s).")
                if epochs_no_improve >= patience:
                    print("Early stopping triggered.")
                    break
    
    scheduler.step()
    
    if epochs_no_improve >= patience:
        print("Training halted due to no improvement.")
        break
    
    print()


Epoch 1/25
----------


Train Phase: 100%|██████████| 449/449 [17:41<00:00,  2.37s/it]


Train Loss: 1.9860 Acc: 0.5331


Val Phase: 100%|██████████| 95/95 [03:03<00:00,  1.93s/it]


Val Loss: 0.6241 Acc: 0.8083
Best model updated.

Epoch 2/25
----------


...

Val Loss: 0.4155 Acc: 0.8744
No improvement for 4 epoch(s).

Epoch 22/25
----------


Train Phase: 100%|██████████| 449/449 [17:44<00:00,  2.37s/it]


Train Loss: 0.6843 Acc: 0.8068


Val Phase: 100%|██████████| 95/95 [03:06<00:00,  1.96s/it]

Val Loss: 0.4218 Acc: 0.8724
No improvement for 5 epoch(s).
Early stopping triggered.
Training halted due to no improvement.


In [ ]:
# Load the best model weights
model.load_state_dict(torch.load("/tmp/best_resnet_model.pth"))
model.eval()

running_loss = 0.0
running_corrects = 0

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing Phase"):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

test_loss = running_loss / len(test_loader.dataset)
test_acc = running_corrects.double() / len(test_loader.dataset)

print(f"Test Loss: {test_loss:.4f} Acc: {test_acc:.4f}")


/root/.ipykernel/1168/command-2724363180960054-1416064897:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/tmp/best_resnet_model.pth"))
Te

Test Loss: 0.3824 Acc: 0.8822


In [ ]:
# Save the trained model
torch.save(model.state_dict(), "/tmp/final_resnet_model.pth")
print("Final model saved.")


Final model saved.


In [ ]:
sample_image_dbfs_path = 'dbfs:/mnt/dog_datasets/test/n02085620-Chihuahua/n02085620_2973.jpg'
# Define the predict_image function (assuming it's already defined)
def predict_image(model, image_path_dbfs, transform, label_mapping):
    """
    Predicts the breed of a dog in the specified image.

    Args:
        model (nn.Module): Trained ResNet model.
        image_path_dbfs (str): DBFS path to the image.
        transform (callable): Transformations to apply to the image.
        label_mapping (dict): Mapping from breed names to integer labels.

    Returns:
        str: Predicted breed name.
    """
    # Copy the image from DBFS to the local filesystem
    local_path = "/tmp/sample_image.jpg"
    dbutils.fs.cp(image_path_dbfs, f"file:{local_path}")

    # Open and transform the image
    image = Image.open(local_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension

    # Move the image tensor to the appropriate device
    image = image.to(device)

    # Perform prediction
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)
        predicted_idx = preds.item()

    # Reverse the label_mapping to get breed name from index
    idx_to_breed = {idx: breed for breed, idx in label_mapping.items()}
    predicted_breed = idx_to_breed.get(predicted_idx, "Unknown")

    return predicted_breed


# Make a prediction
predicted_breed = predict_image(model, sample_image_dbfs_path, val_test_transforms, label_mapping)
print(f"Predicted Breed: {predicted_breed}")


Predicted Breed: n02085620-Chihuahua


In [ ]:
import mlflow
import mlflow.pyfunc
import mlflow.pytorch
import torch
from torch import nn
from torchvision import models, transforms
from PIL import Image
import io
import base64
import pandas as pd
import json
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


class ResNetClassifier(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(ResNetClassifier, self).__init__()
        self.model = models.resnet50(pretrained=pretrained)
        
        for param in self.model.parameters():
            param.requires_grad = False
        
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)
        
    def forward(self, x):
        return self.model(x)

num_classes = len(label_mapping)
model = ResNetClassifier(num_classes=num_classes, pretrained=False)
model.load_state_dict(torch.load("/tmp/final_resnet_model.pth", map_location=device))
model = model.to(device)
model.eval()

class DogBreedClassifierWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, model, label_mapping):
        self.model = model
        self.label_mapping = label_mapping
        self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
        self.idx_to_breed = {idx: breed for breed, idx in label_mapping.items()}
    
    def load_context(self, context):
        pass
    
    def predict(self, context, model_input):
        image_base64 = model_input['image'].iloc[0]
        image_bytes = base64.b64decode(image_base64)
        image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
        image = self.transform(image).unsqueeze(0).to(device)
        
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(image)
            _, preds = torch.max(outputs, 1)
            predicted_idx = preds.item()
        
        predicted_breed = self.idx_to_breed.get(predicted_idx, "Unknown")
        return predicted_breed


sample_image_path = "/tmp/sample_image.jpg"


try:
    files = dbutils.fs.ls("dbfs:/mnt/dog_datasets/test/n02085620-Chihuahua/")
    dbfs_image_path = files[0].path
    print(f"Using image: {dbfs_image_path}")
except Exception as e:
    print("Error accessing the directory or files.")
    print(f"Error: {e}")

dbutils.fs.cp(
    dbfs_image_path,
    f"file:{sample_image_path}"
)

# Read and encode the image
with open(sample_image_path, "rb") as f:
    image_bytes = f.read()
    image_base64 = base64.b64encode(image_bytes).decode('utf-8')

input_example = pd.DataFrame({"image": [image_base64]})

input_schema = Schema([ColSpec("string", "image")])
output_schema = Schema([ColSpec("string")])
signature = ModelSignature(inputs=input_schema, outputs=output_schema)

wrapped_model = DogBreedClassifierWrapper(model=model, label_mapping=label_mapping)

with mlflow.start_run(run_name="DogBreedClassifierDeployment"):
    mlflow.pyfunc.log_model(
        artifact_path="dog_breed_classifier_model",
        python_model=wrapped_model,
        signature=signature,
        input_example=input_example
    )
    
    with open("/tmp/label_mapping.json", "w") as f:
        json.dump(label_mapping, f)
    mlflow.log_artifact("/tmp/label_mapping.json", artifact_path="label_mapping")
    
    print("Model and artifacts logged to MLflow.")
    
    run_id = mlflow.active_run().info.run_id

result = mlflow.register_model(
    f"runs:/{run_id}/dog_breed_classifier_model",
    "DogBreedClassifierModel"
)
print(f"Model registered with name: {result.name} and version: {result.version}")

from mlflow.tracking import MlflowClient
client = MlflowClient()
client.set_registered_model_alias(
    name=result.name,
    alias="Production",
    version=result.version
)
print(f"Alias 'Production' set for model '{result.name}' version {result.version}.")

print(f"Model version {result.version} transitioned to 'Production' stage.")


Using device: cpu


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/root/.ipykernel/1168/command-2724363180960057-2366911796:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details).

Using image: dbfs:/mnt/dog_datasets/test/n02085620-Chihuahua/n02085620_11948.jpg


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

2024/10/06 05:07:39 INFO mlflow.store.artifact.cloud_artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


...

Registered model 'DogBreedClassifierModel' already exists. Creating a new version of this model...


2024/10/06 05:07:43 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

2024/10/06 05:07:45 INFO mlflow.store.artifact.cloud_artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
Created version '2' of model 'find_my_pet.default.dogbreedclassifiermodel'.


Model registered with name: find_my_pet.default.dogbreedclassifiermodel and version: 2
Alias 'Production' set for model 'find_my_pet.default.dogbreedclassifiermodel' version 2.
Model version 2 transitioned to 'Production' stage.


In [ ]:
torch.save(model.state_dict(), '/dbfs/mnt/dog_datasets/final_resnet_model.pth')
